# Selecting Rows and Columns

Before we go into statistical and inferential methods in cleaning data, we will start with some more basic data cleaning operations. When you import a dataset you may judge certain columns and rows are just not needed, or you want to narrow your focus to a specific scope. We will use this section to learn different methods for selecting rows and columns using indexing and conditional methods. 

First let's declare a dataframe of some users. 

In [ ]:
import pandas as pd 

df = pd.DataFrame({
    "username" : ["thomasnield","samiam", "joecool"],
    "first_name": ["Thomas", 'Sam', 'Joe'], 
    "last_name": ["Nield", 'Scala', 'Morrison'], 
    "email": ["tmnield@outlook.com", 'sam.scala@gmail.com', 'joe@rexonmetals.com']
})

df

## Understanding iloc and loc 

There are two critical functions to know in Pandas when you are selecting by index: `loc` and `iloc`. It is very easy to confuse these two, as the first works on labels and the second for numeric indices. 

Here is where people get confused. Let's say we want to select the first record. We can use both `loc` and `iloc` to do this, and they both produce the same answer.

> Remember that Python and Pandas uses 0-based indexing, meaning the first element will start at index 0 rather than index 1! 

In [ ]:
df.iloc[0]

In [ ]:
df.loc[0]

It seems `loc` and `iloc` do not behave any differently, and this is where people get tripped up. Let's change the index to be the `username`. 

In [ ]:
df.set_index('username', inplace=True)

Now try to run `loc` and `iloc` again. Notice that `loc` works fine, but `iloc` no longer does!

In [ ]:
df.iloc[0]

In [ ]:
df.loc[0] # this will cause an error 

This is because `iloc` looks up a row by a numeric index, and that is what you should use if that is your intent. The `loc` uses the labelled index which earlier (by default) is also a numeric index, but we then changed it to the `username`. 

Therefore, if we looked up by an actual `username` value such as "thomasnield" then the `loc` function will work. 

In [ ]:
df.loc['thomasnield']

## Selecting Ranges 

We can also look up multiple rows at multiple indices, whether they are numeric or labels. We can use the Python range operator `:` to get a range of numeric positions or labels (if the labels have ordering behavior). Example: I can get the first and second rows.

In [ ]:
df.iloc[0:2]

If you expected the third row to be included because it has an index of 2, and we selected range `0:2`, this is why it was not included. The end of the range is exclusive and omits that last element in the selection. Another way to think of it is we are selecting the indices *between* each digit. This is usually helpful for me and here is a visual to demonstrate grabbing the first two elements of a collection. 

svg image

Whenever there is a 0 in a range, we can omit it and it will be implied.

In [ ]:
df.iloc[:2]

If we leave the end value off as well, that will extend to the end of the range. Below we grab everything from the second record and after. 

In [ ]:
df.iloc[1:]

If we provide just a brackets with a colon inside, it will select all rows. This may seem pointless, but it will serve as a placeholder when we select columns shortly. 

In [ ]:
df.iloc[:]

Now we can provide a second range to get certain columns, but already specifying to include all rows. Below we grab all rows and the second through third columns. 

In [ ]:
df.iloc[:, 1:3]

### Negative Index

We can also use a negative index to grab rows or columns from the opposite direction, like grabbing the last two columns.

In [ ]:
df.iloc[:,-2:]

Another negative index example: using `-1` to specify grabbing the last row or the last column. 

In [ ]:
df.iloc[-1]

In [ ]:
df.iloc[:,-1]

## Picking Rows and Columns

To get extra picky, we can provide a list of indices instead of a range to pick only certain columns or certain rows. Below get the second and third row, and the first and third columns. 

In [ ]:
df.iloc[1:3, [0,2]]

There is a `loc` equivalent to this cherrypicking as well, where we can provide a list of labels we are interested in. Below I grab the rows with usernames `samiam` and `thomasnield` then extract the `email` column.

In [ ]:
df.loc[["samiam","thomasnield"], "email"]

## Resetting the Index

You can reset the index back its default behavior by calling `reset_index()`. Make sure to use the `inplace=True` argument so it replaces the existing DataFrame rather than create a new one. 

In [ ]:
df.reset_index(inplace=True)

> Similar to `loc` and `iloc` there is also an `at` and `iat`. These return a single value at a specific row and column index using numeric or labelled indices respectively.

## Dropping Rows by Condition

Let's look at how to filter out rows and columns. There are multiple ways to do this. Let's talk about logical operators first. 

Notice how we can extract a column, use the `str` property, and get string-related methods. Let's use `startswith()` and find usernames that begin with the letter "s."

In [ ]:
df["username"].str.startswith("s")

The result might not be something you expect. We got a series of boolean `True/False` values indicating whether that value matches that condition. 

You might be wanting to simply list records that evaluated to `True`. We can achieve that by passing that series of `True/False` values back into the DataFrame and then it will only yield records that match `True` in that index. 

In [ ]:
condition = df["username"].str.startswith("s")

df[condition]

You can also just embed that logical expression inside the DataFrame getter brackets. 

In [ ]:
df[df["username"].str.startswith("s")]

We can also use the `&` and `|` to perform *and* and *or* operations respectively with two or more conditions. 

In [ ]:
df[df["username"].str.startswith("s") & df["email"].str.contains("gmail")]

# Dropping Columns and Rows


There will be times you want to drop rows and columns that are not needed for your task. This is what the `drop()` function is for. 

Below I drop the first and second rows from my DataFrame. Because I want to drop rows, I specify the `axis=0`. 

In [ ]:
df.drop([0,1], axis=0)

> As always, while not being done here, use `inplace=True` if you want it to replace the existing DataFrame.

Note carefully that this uses the index. Therefore if you have different labels than a typical numeric index, you will need to specify with those labels.

Below I set the index of my DataFrame to use the `username` and then drop those two rows by those usernames. 

In [ ]:
df.set_index("username").drop(["thomasnield","samiam"], axis=0)

You can also use `drop()` to remove columns. Below I specify columns by `axis=1` and drop the `username` and `email` columns from the DataFrame.

In [ ]:
df.drop(["username", "email"],axis=1)

If you want to drop columns by a numeric index, you will need to retrieve that column name by grabbing the `columns` index object. Below we delete the first and fourth columns in our DataFrame by looking up their corresponding column labels, and then packaging it into a list. 

In [ ]:
num_indices = [0,3]
df.drop([df.columns[i] for i in num_indices], axis=1)

## Appending Rows and Columns 

### Appending Columns

Appending a column to a DataFrame can be done in several ways. The simplest is to define the new column label inside the square brackets like `df["phone"]` and then assign a simple list, a dictionary, a Series, or another DataFrame. 

Below we create a new `phone` column and apply the data using a simple list. The number of values must match the number of records. 

In [ ]:
df["phone"] = ["213-247-5724","754-238-8237","233-555-2311"]

df


If you want to add a column at a specific location, you can use the `insert()` function. Provide first the positional index and the column name, and then a list of values. 

Below we add a `twitter` column in the fourth column position of the DataFrame.

In [ ]:
df.insert(3, "twitter", ["@thomasnield76","@samiam46","@joe564"])

df

### Appending Rows

Adding rows can be done similarly as adding columns, but uses the `loc` property. You will need to provide the index label for that new record and assign it. 

In [ ]:
df.loc[3] = ["jasonmarley", "Jason", "Marley", "@OtherJason45", "jason@marlyco.net", "214-282-9998"]
df

You can also use the `concat()` function to append one or more dataframes together.

In [ ]:
new_record = pd.DataFrame({
    "username":["wyatt_urp"],
    "first_name":["Wyatt"],
    "last_name":["Jones"], 
    "email":["wyatt.jones@jonesco.com"],
    "twitter": [None],
    "phone":["444-244-7642"]
})
df = pd.concat([df, new_record])

df

## Updating Data 

### Updating a Column

You can update an entire column in Pandas by using the `=` operator. Below we update all the `email` values to be uppercase. 

In [ ]:
df["email"] = df["email"].str.upper()

df

### Updating On a Condition 

We can also conditionally update one or more specific records by passing a logical condition to the `loc` function as well as the desired column to be updated. 

Below we change joe's email address. 

In [ ]:
condition = df["first_name"].eq("Joe")

df.loc[condition, "email"] = "joe@gmail.com"

df

### Updating a Row

You can also use an index for a row, and assign a new record for that row. 

In [ ]:
df.loc[1] = ['samiam2022','Samuel','Scala',None,'sam.scala@gmail.com',None]

df

## Melting

There will be situations in data cleaning where you will want to unpivot columns to become row values. For example, take this simple but possibly inconvenient DataFrame that breaks up a revenue amount by category, but also by the years `2022` and `2021` as two separate columns. 

In [ ]:
df = pd.DataFrame({
    "CATEGORY" : ["ALPHA", "BETA", "GAMMA"],
    "2022" : [120, 250, 280],
    "2021" : [320, 420, 170]
})

df

This is not exactly normalized data, where we would expect a `YEAR` column and then `2022` and `2021` would be values in that column. 

Thankfully, the `melt()` function in Pandas will do this transformation. Let's take a look and see if you can dissect how this function is used, and what its arguments mean.

In [ ]:
df.melt(id_vars=['CATEGORY'], value_vars=['2022','2021'], var_name="YEAR", value_name="AMOUNT")

As you will see above, the `id_vars` is essentially the columns that will be unchanged and "anchor" the records. But the `value_vars` will be the column names that will be flipped into values for a column we name `YEAR`, which is also an argument `var_name`. The `value_name` argument will provide a name for that column all the values will be moved to. 

## Exercise 

Complete the code below so the `employee_id` is the index for the dataframe. Make the `department` field uppercase, and then add a column `birth_date` with values `1985-04-02`, `1987-05-16`, and `1979-01-03`. Then delete the record where the `employee_id` is `c275213`. 

In [ ]:
import pandas as pd 

df = pd.DataFrame({
    'employee_id' : ['e947521','e875624','c275213'],
    'hire_date' : ['2019-12-01','2018-05-21','2018-11-03'],
    'department' : ['marketing', 'research','accounting']
})

# set the index 
df.?(?, inplace=True)

# make 'department' column uppercase
df['department'] = ?

# add birth_date column
? = ['1985-04-02', '1987-05-16', '1979-01-03']

# drop record with 'employee_id' of c275213
df.?(?, inplace=True)

# display the DataFrame
df

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd 

df = pd.DataFrame({
    'employee_id' : ['e947521','e875624','c275213'],
    'hire_date' : ['2019-12-01','2018-05-21','2018-11-03'],
    'department' : ['marketing', 'research','accounting']
})

# set the index 
df.set_index('employee_id', inplace=True)

# make 'department' column uppercase
df['department'] = df['department'].str.upper()

# add birth_date column
df['birth_date'] = ['1985-04-02', '1987-05-16', '1979-01-03']

# drop record with 'employee_id' of c275213
df.drop(["c275213"], inplace=True)

# display the DataFrame
df